In [1]:
import tensorflow as tf
import tensorlayer as tl

/home/jing/anaconda2/lib/python2.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
graph = tf.get_default_graph()
config = tf.ConfigProto()
session = tf.Session(graph=graph, config=config)

In [ ]:
scan = tf.placeholder(
            tf.float32, [None, self.obs_shape[1], self.obs_shape[0]],
            'scan_ph')
        goal = tf.placeholder(tf.float32, [None, 2], 'goal_ph')
        vel = tf.placeholder(tf.float32, [None, 2], 'vel_ph')

        net = tl.layers.InputLayer(scan, name='scan_input')
        net = tl.layers.MeanPool1d(net, filter_size=3, strides=2, name='min_pooling1')
        net = tl.layers.MeanPool1d(net, filter_size=3, strides=2, name='min_pooling2')
        net = tl.layers.MeanPool1d(net, filter_size=3, strides=2, name='min_pooling3')

        net = tl.layers.Conv1dLayer(
            net, act=tf.nn.relu, shape=[5, self.obs_shape[0], 8], stride=2,
            name='cnn1')
        net = tl.layers.Conv1dLayer(
            net, act=tf.nn.relu, shape=[3, 8, 16], stride=2,
            name='cnn2')
        net = tl.layers.FlattenLayer(net, name='fl')
        net = tl.layers.DenseLayer(
            net, n_units=128, act=tf.nn.relu, name='cnn_output')
        cnn_output = net.outputs

        act_net = tl.layers.InputLayer(
            tf.concat([goal, vel, cnn_output], axis=1), name='goal_input')
        act_net = tl.layers.DenseLayer(
            act_net, n_units=64, act=tf.nn.tanh, name='act1')
        act_net = tl.layers.DenseLayer(
            act_net, n_units=64, act=tf.nn.tanh, name='act2')
        linear = tl.layers.DenseLayer(
            act_net, n_units=1, act=tf.nn.sigmoid, name='linear')
        angular = tl.layers.DenseLayer(
            act_net, n_units=1, act=tf.nn.tanh, name='angular')
        with tf.variable_scope('means'):
            action_mean = tf.concat([linear.outputs, angular.outputs], axis=1)